In [ ]:
%matplotlib  inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys
from pathlib import Path

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from copy import deepcopy

from proteomics_preprocessing import *
from utils.proteomics_utils import *
from mhc_analysis_utils import *

from modelv1 import Model    

from sklearn.metrics import roc_auc_score

# Data Requirements

## MHC I data
### MHCFlurry18 (BA) [ODonnell2018]
Download curated MHC I dataset by [ODonnell2018] from https://data.mendeley.com/datasets/8pz43nvvxh/1/files/e1916ecf-b544-40e6-b1fe-e0024bea76a7/data_curated.20180219.tar.bz2?dl=1 and extract the archive in ./data

### NetMhCpan4.0 (MS) [Jensen2018]
From http://www.cbs.dtu.dk/suppl/immunology/NetMHCpan-4.0/ download 0th CV split files f000_ba (train) and c000_ba (val, 20%) store in data_dir/NetMHCpan_4_0_train.
            

## MHC II data

### NetMHCIIpan3.2 (BA) [Jurtz2017]
From www.cbs.dtu.dk/suppl/immunology/NetMHCIIpan-3.2/ download 1st CV split files train1 (train) and test1 (val) and store as train1.txt and test1.txt in ./data/NetMHCIIpan_3_2_train.
            
### NetMHCIIpan4.0 (MS) [Reynisson2020]
Download http://www.cbs.dtu.dk/suppl/immunology/NAR_NetMHCpan_NetMHCIIpan/NetMHCIIpan_train.tar.gz, unpack and save as NetMHCIIpan_train in ./data.

## Test data
### Stability measurements for sars-cov-2 peptides [Prachar 2020]
Download https://www.immunitrack.com/wp/wp-content/uploads/Covid19-Intavis-Immunitrack-datasetV2.xlsx and save in ./data directory.


**References**
- [ODonnell2018] T. J. O’Donnell, A. Rubinsteyn, M. Bonsack, A. B. Riemer, U. Laserson, and J. Hammerbacher, “MHCflurry: Open-Source Class I MHC Binding Affinity Prediction,” Cell Systems, vol. 7, no. 1, pp. 129–132.e4, Jul. 2018. [Online].
Available: https://doi.org/10.1016/j.cels.2018.05.014

- [Jurtz2017] Vanessa Jurtz, Sinu Paul, Massimo Andreatta, Paolo Marcatili, Bjoern Peters, and Morten Nielsen, NetMHCpan-4.0:  Improved peptide-MHC class I interaction predictions integrating eluted lig-and and peptide binding affinity data.Journal of immunology (Baltimore, Md. :  1950), 199(9):3360–3368, Nov 2017. ISSN 1550-6606. Available: https://doi.org/10.4049/jimmunol.1700893

- [Jensen2018] Kamilla Kjærgaard Jensen, Massimo Andreatta, Paolo Marcatili, Søren Buus, Jason A. Greenbaum,Zhen Yan, Alessandro Sette, Bjoern Peters, and Morten Nielsen, Improved methods for predictingpeptide binding affinity to MHC class II molecules.Immunology, 154(3):394–406, 2018. Available: https://doi.org/10.1111/imm.12889

- [Reynisson2020] Birkir Reynisson,  Bruno Alvarez,  Sinu Paul,  Bjoern Peters,  and Morten Nielsen,   NetMHCpan-4.1 and NetMHCIIpan-4.0:  improved predictions of MHC antigen presentation by concurrentmotif deconvolution and integration of MS MHC eluted ligand data.Nucleic Acids Research,48(W1):W449–W454, 05 2020.   ISSN 0305-1048.   doi:  10.1093/nar/gkaa379. Available: https://doi.org/10.1093/nar/gkaa379

- [Prachar2020] Marek  Prachar,  Sune  Justesen,  Daniel  Bisgaard  Steen-Jensen,  Stephan  Thorgrimsen,  Erik  Jurgons,  Ole Winther, and Frederik OtzenBagger, Identification and validation of 174COVID-19 vaccine candidate epitopes reveals low performance of common epitope predictiontools. Scientific Reports, 10(1), Nov. 2020.  Available: https://doi.org/10.1038/s41598-020-77466-4


# Data Preprocessing

## Binding affinity dataset
Create a directory for each dataset with subdirectories for each allele. The output of the preprocessing saved in each allele subdirectory is structured as follows:

- *tok.npy* sequences as list of numerical indices (mapping is provided by *tok_itos.npy*)
- *label.npy* label as list of binding affintiy values (mapping is provided by *label_itos.npy*)
- *train_IDs.npy/val_IDs.npy/test_IDs.npy* numerical indices identifying training/validation/test set by specifying rows in tok.npy
- *train_IDs_prev.npy/val_IDs_prev.npy/test_IDs_prev.npy* original non-numerical IDs for all entries that were ever assigned to the respective sets (used to obtain consistent splits for downstream tasks)
- *ID.npy* original non-numerical IDs for all entries in tok.npy


In [ ]:
dataset_path= Path("./reg_sars_cov_BA")
dataset_path.mkdir(exist_ok=True)

# sheet titles in Covid19-Intavis-Immunitrack-datasetV2.xlsx corresponding to allele names 
# no BA data available for "10 C0102"
allele_sheets = ["1 A0101",
"2 A0201",
"3 A0301",
"4 A1101",
"5 A2402",
"6 B4001",
"7 C0401",
"8 C0701",
"9 C0702",

"11 DRB10401"]

prep = Preprocess()
for allele in allele_sheets:
    prep.clas_mhc_sars_cov(allele, "flurry", working_folder=dataset_path/allele, pretrained_folder="../../USMPep/git_data/lm_netchop_peptides")

## Mass spectroscopy dataset

In [ ]:
dataset_path= Path("./reg_sars_cov_MS")
dataset_path.mkdir(exist_ok=True)

# sheet titles in Covid19-Intavis-Immunitrack-datasetV2.xlsx corresponding to allele names
# no MS data available for "8 C0701" and "10 C0102"
allele_sheets = ["1 A0101",
"2 A0201",
"3 A0301",
"4 A1101",
"5 A2402",
"6 B4001",
"7 C0401",
                 
"9 C0702",

"11 DRB10401"]

prep = Preprocess()
for allele in allele_sheets:
    prep.clas_mhc_sars_cov(allele, "netmhcpan4", MS=True, working_folder=dataset_path/allele, pretrained_folder="../../USMPep/git_data/lm_netchop_peptides")

# Downstream Training and Evaluation

from_scratch is set to True for training from scratch and set to False for using a language model. By default we will use the provided language model that was pretrained on Netchop peptides (../git_data/lm_netchop_peptides)

## Train ensemble on BA data

In [ ]:
alleles = ["1 A0101",
            "2 A0201",
            "3 A0301",
            "4 A1101",
            "5 A2402",
            "6 B4001",
            "7 C0401",
            "8 C0701",
            "9 C0702",

            "11 DRB10401"]

data_dir = "./reg_sars_cov_BA"

modelv1 = Model()

allele_dir_list = [data_dir +"/" + a for a in alleles]
n_alleles = len(allele_dir_list)

for ensemble_i in range(10):
    for clas_folder in allele_dir_list:
        # Train
        # model_filename_prefix should end on the ensemble index ensemble_i
        modelv1.generic_model(working_folder=clas_folder, model_filename_prefix="USMPep_LM_BA_{}".format(ensemble_i),
                        from_scratch=False, 
                        pretrained_folder="../git_data/lm_netchop_peptides",pretrained_model_filename="lm_1l_3_enc",
                        eval_on_test=True, export_preds=True,
                        train=True,clas=True, regression=True, concat_train_val=True,
                        emb_sz=50,nh=64,nl=1,
                        bs=32, epochs=10, lr=0.05,
                        wd=1e-7, dropout=0.1,
                        interactive=False,
                        metrics=[])
        
    # be careful to give the correct preds_filename. If eval_on_test=True, the test predictions are saved as "preds_valid.npz",
    # if eval_on_val_test=True, the test predictions are saved as "preds_test.npz",
    # val_on_test=True, the test predictions are saved as "preds_valid.npz"    
    collect_preds_npz(data_dir, n_alleles, subfoldername="", ensemble_i=ensemble_i, preds_filename='preds_valid.npz', 
                    ranked_alleles=False, allele_list=alleles)        


## Train ensemble on MS data

In [ ]:
alleles = ["1 A0101",
            "2 A0201",
            "3 A0301",
            "4 A1101",
            "5 A2402",
            "6 B4001",
            "7 C0401",
           
            "9 C0702",

            "11 DRB10401"]

data_dir = "./reg_sars_cov_MS"

modelv1 = Model()

allele_dir_list = [data_dir +"/" + a for a in alleles]
n_alleles = len(allele_dir_list)

for ensemble_i in range(5):
    for clas_folder in allele_dir_list:
        # Train
        # model_filename_prefix should end on the ensemble index ensemble_i
        modelv1.generic_model(working_folder=clas_folder, model_filename_prefix="USMPep_LM_MS_{}".format(ensemble_i),
                        from_scratch=False, 
                        pretrained_folder="../git_data/lm_netchop_peptides",pretrained_model_filename="lm_1l_3_enc",
                        eval_on_test=True, export_preds=True,
                        train=True,clas=True, regression=False, concat_train_val=True,
                        emb_sz=50,nh=64,nl=1,
                        bs=512, epochs=10, lr=0.05,
                        wd=1e-7, dropout=0.1,
                        interactive=False,
                        metrics=[])
        
    # be careful to give the correct preds_filename. If eval_on_test=True, the test predictions are saved as "preds_valid.npz",
    # if eval_on_val_test=True, the test predictions are saved as "preds_test.npz",
    # val_on_test=True, the test predictions are saved as "preds_valid.npz"    
    collect_preds_npz(data_dir, n_alleles, regression=False, subfoldername="", ensemble_i=ensemble_i, preds_filename='preds_valid.npz', 
                    ranked_alleles=False, allele_list=alleles)        


# Evaluate

To evaluate the predictions after an ensemble of models has been trained, load the *.npz files and compile model names, predictions, targets, sequences and allele names in a DataFrame.

In [ ]:
# Load predictions and create ensembles
results = []
data_dirs = ("./reg_sars_cov_BA", "./reg_sars_cov_MS")

agg_dic = {"targs": lambda x: x.iloc[0], "preds":np.mean}

for data_dir in data_dirs:
    # load predictions for all alleles per ensemble
    ensemble = []
    for f in Path(data_dir).glob("*.npz"):
        if "BA" in data_dir:
            df_tmp = read_npz(f, ensemble=True,transform_targs=False, transform_preds=False, with_args=False)
            ensemble.append(df_tmp)
        elif "MS" in data_dir and "MS" in str(f):
            df_tmp = read_npz(f, ensemble=True,transform_targs=False, transform_preds=False, sigmoid=False,
                  with_args=False)
            df_tmp["preds"] = 1- df_tmp["preds"] 
            ensemble.append(df_tmp)

    # aggregate ensembles
    ensemble = pd.concat(ensemble, ignore_index=True, sort=True)

    # keep single model predictions
    single = ensemble.copy()[['rank', 'ID', 'targs', 'preds', 'model']]
    single = single.set_index(["model", "rank", "ID"])

    # simply average predicitons for an ensemble predictor
    ensemble["model"] = ensemble["model"].apply(lambda x: x[:-2])
    ensemble= ensemble.groupby(["model", "rank", "ID"]).agg(agg_dic)

    result = pd.concat([ensemble, single], sort=True)
    
    results.append(result)
    
results = pd.concat(results)

# Create ensemble of BA regressors and MS classifiers (simple average)
bams_ens_models = ["USMPep_LM_BA_{}".format(i) for i in range(5)] + ["USMPep_LM_MS_{}".format(i) for i in range(5)] 
# drop allele C0701 because no MS data is available
bams_ens = results.loc[bams_ens_models].drop("8 C0701", axis=0, level=1).reset_index()
bams_ens["model"] = "USMPep_LM_BAMS"
bams_ens = bams_ens.groupby(["model", "rank", "ID"]).agg(agg_dic)
results = pd.concat([results, bams_ens], sort=True)

results = results.loc[["USMPep_LM_BA","USMPep_LM_BAMS"]]

In [ ]:
def aucroc_stability_targs(df, stability_threshold=60):
    targs = (df["targs"]>stability_threshold)*1.0
    if targs.sum()>0 and targs.mean()<1:
        return roc_auc_score((df["targs"]>stability_threshold)*1.0, df["preds"])
    else:
        return np.nan
    
scores = pd.DataFrame({"AUC ROC":results.groupby(level=[0,1]).apply(aucroc_stability_targs),
                      "Spearman r":results.groupby(level=[0,1]).apply(spearmanr_eval, filter_quantitative=False)})
           
scores["AUC ROC"].unstack().T.plot.bar(title="AUC ROC")

print("median Spearman r over alleles A*01:01, A*02:0, A*03:01, A*11:01, A*24:02, B*40:01, C*04:01, C*07:02, C*01:02:")
print(scores["Spearman r"].drop(["8 C0701", "11 DRB10401"], axis=0, level=1).groupby(level=0).median())
print("\n")
print("median AUC ROC over alleles A*01:01, A*02:0, A*03:01, A*11:01, A*24:02, B*40:01:")
print(scores["AUC ROC"].drop(["7 C0401",
                                "8 C0701",
                                "9 C0702",
                                "10 C0102",
                                "11 DRB10401"], axis=0, level=1).groupby(level=0).median())